In [1]:
cd /home

/home


# Astromer I vs Astromer II

In [50]:
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np
import os

from sklearn.metrics import r2_score, mean_squared_error
from presentation.experiments.utils import get_clf_summary
from src.utils import get_metrics

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.sans-serif": ["Helvetica"]})
# for Palatino and other serif fonts use:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


**WARNING**: This notebook was made to visualize classification results only. 

All the following blocks assume you already ran the script `./presentation/experiments/astromer_{1/2}/script.py`.

The directory looks like: 
- `presentation`
    - `astromer_1/results/<experiment_folder>`
        - `finetuning`
        - `classification`
        - `pretraining`
    - `astromer_2/results/<experiment_folder>`
        - `finetuning`
        - `classification`
        - `pretraining`

In [51]:
folders = [
    './presentation/experiments/astromer_1/results/v1/',
    './presentation/experiments/astromer_2/results/v2/'
]
tags = ['Astromer I', 'Astromer II']

In [55]:
summary = []
for folder, tag in zip(folders, tags):
    partial = get_clf_summary(folder, tag)
    summary.append(partial)
    
summary = pd.concat(summary, axis=0)

In [59]:
for tag, frame_0 in summary.groupby('tag'):
    for downstream_data, frame_1 in frame_0.groupby('downstream_data'):
        for (spc, clfname), frame_2 in frame_1.groupby(['samples_per_class', 'clf_name']):
            print(clfname)

att_mlp
att_mlp
att_mlp
att_mlp
all_mlp
att_mlp
cls_mlp
all_mlp
att_mlp
cls_mlp
all_mlp
att_mlp
cls_mlp
all_mlp
att_mlp
cls_mlp


In [43]:
df.columns

Index(['exp_name', 'data', 'checkpoint', 'gpu', 'debug', 'encoder_mode',
       'num_layers', 'num_heads', 'head_dim', 'pe_dim', 'pe_base', 'pe_exp',
       'mixer', 'dropout', 'avg_layers', 'lr', 'bs', 'patience', 'num_epochs',
       'window_size', 'probed', 'rs', 'clf_val_acc', 'clf_val_loss',
       'clf_test_precision', 'clf_test_recall', 'clf_test_f1', 'clf_test_acc',
       'clf_name', 'downstream_data', 'samples_per_class'],
      dtype='object')

In [44]:
subgroups = df.groupby(['downstream_data', 'samples_per_class'])

In [46]:
for k, subframe in subgroups:
    print(subframe['clf_test_f1'])

7      0.05086877000457247
9     0.053827333100314574
11    0.054773633591367575
Name: clf_test_f1, dtype: object
6      0.05600448933782267
8     0.053714684339030344
10    0.046376811594202906
Name: clf_test_f1, dtype: object
0    0.36860070779541826
2    0.36923832859135086
4     0.4068101669594008
Name: clf_test_f1, dtype: object
1    0.35045985893186926
3     0.3905066386329509
5     0.3513379652324876
Name: clf_test_f1, dtype: object
